ESQUELETO:

Este es el archivo para subir el modelo a la nube.


CÓMO FUNCIONA:

1. Tomar un modelo ya realizado.

2. Lo sube a la nube.

ASEGÚRENSE DE QUE SE COORDINAN LO NECESARIO PARA QUE LOS NOMBRES COINCIDAN (no es de chill cambiar el estándar xd).

In [ ]:
## STEP 1: Set up the workspace.

import json

#GETTING MY ID:
#id = open('id_Confidential.json', 'r')
#mi = json.load(id)

from azureml.core import Workspace

ws = Workspace.create(name="workspace",
                      subscription_id = "id", ## Borre este para que no tenga el id confidencial
                      resource_group = "resource_group2",
                      location = "centralindia")

from azureml.core.model import Model

mname = "model"
registered_model = Model.register(model_path="model.pkl",
                                  model_name=mname,
                                  workspace=ws)

c:\Users\pamec\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


Performing interactive authentication. Please follow the instructions on the terminal.


The default web browser has been opened at https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.


Interactive authentication successfully completed.


Cleaning up past default Resource Group Deployments on the subscription to avoid limit of 10
Deleting past Resource Group Deployment with name: DeployResourceGroup-2deb444159
Deploying KeyVault with name workspackeyvaultd52e1b03.
Deploying StorageAccount with name workspacstorage1b43ca60d.
Deploying AppInsights with name workspacinsightsd42e6836.
Deployed AppInsights with name workspacinsightsd42e6836. Took 35.61 seconds.
Deploying Workspace with name workspace.
Deployed Workspace with name workspace. Took 42.75 seconds.
Registering model model


In [4]:
################ score.py ###################

scorepy = f"""
import json
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model

def init():
  global model
  model_path = Model.get_model_path('{mname}')
  model = joblib.load(model_path)


def run(raw_data):
  try: ## Try la predicción.
    data = json.loads(raw_data)['data'][0]
    data = pd.DataFrame(data)

    result = model.predict(data).tolist()

    return json.dumps(result)
  except Exception as e:
    return json.dumps(str(e))
"""

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()

In [5]:
## STEP 2: Inference configuration: the blueprints for the German carpenter about how to build your furniture.

from azureml.core.environment import Environment
virtual_env = Environment("env-4-housing")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas','scikit-learn']) ## Asegúrense de coordinarse con el departamento de modelos para incluir los paquetes correctos. Si no, BOOM!


from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1) ## ASEGÚRENSE DE ASIGNAR SUFICIENTE MADERA PARA SUS MUEBLES.

service = Model.deploy(workspace=ws,
                       name='service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

C:\Users\pamec\AppData\Local\Temp\ipykernel_14860\944137596.py:18: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [6]:
service.wait_for_deployment()

scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()

ACI service creation operation finished, operation "Succeeded"
